# Script to convert between geojson (outputted by QGIS) and the json format required by labelme

In [1]:
import json
import os
from datetime import datetime

In [8]:
def geojson_to_labelme(geojson_path, image_path, image_height=None, image_width=None, label_name='waterhole'):
    """
    Convert GeoJSON point features to LabelMe JSON format.
    
    Args:
        geojson_path (str): Path to input GeoJSON file
        image_path (str): Path to the corresponding image file
        
    Returns:
        dict: LabelMe formatted JSON
    """
    # Read GeoJSON file
    with open(geojson_path, 'r') as f:
        geojson_data = json.load(f)
    
    # Initialize LabelMe JSON structure
    labelme_json = {
        "version": "5.6.1",
        "flags": {},
        "shapes": [],
        "imagePath": os.path.basename(image_path),
        "imageData": None,  # LabelMe stores base64 image data here, but we'll leave it empty
        "imageHeight": image_height,  
        "imageWidth": image_width
    }
    
    # Convert each GeoJSON feature to LabelMe shape
    for feature in geojson_data['features']:
        # Get coordinates (assuming Point geometry)
        coords = feature['geometry']['coordinates']
        
        # Create point annotation (LabelMe uses a small rectangle for points)
        point_size = 5  # Size of the point representation in pixels
        x, y = coords[0], coords[1]
        
        shape = {
            "label": label_name,
            # "label": feature['properties'].get('label', 'point'),  # Use 'label' property or default to 'point'
            "points": [
                [x - point_size, y - point_size],  # Top-left
                [x + point_size, y + point_size]   # Bottom-right
            ],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        }
        
        labelme_json['shapes'].append(shape)
    
    # Add creation time
    labelme_json['timeStamp'] = datetime.now().isoformat()
    
    return labelme_json

def save_labelme_json(labelme_json, output_path):
    """Save the LabelMe JSON to file."""
    with open(output_path, 'w') as f:
        json.dump(labelme_json, f, indent=2)

In [9]:
# Example usage
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    geojson_path = "images\labels\waterholes_holly_v1.geojson"
    image_path = "images\labels\20230428_example_SF.png"
    output_path = "images\labels\waterholes_holly_v1.json"
    
    # Convert and save
    labelme_json = geojson_to_labelme(geojson_path, image_path)
    save_labelme_json(labelme_json, output_path)

# Using a csv file as input instead of a geojson file

In [ ]:
import json
import os
from datetime import datetime
import pandas as pd
import numpy as np

def csv_to_labelme(x, y, labels=None, image_path=None, image_height=None, image_width=None):
    """
    Convert coordinate columns from a DataFrame to LabelMe JSON format.
    
    Args:
        x (pd.Series): Series containing x/longitude coordinates
        y (pd.Series): Series containing y/latitude coordinates
        labels (pd.Series, optional): Series containing point labels. Defaults to None
        image_path (str, optional): Path to the corresponding image file. Defaults to None
        image_height (int, optional): Height of the image in pixels. Defaults to None
        image_width (int, optional): Width of the image in pixels. Defaults to None
        
    Returns:
        dict: LabelMe formatted JSON
    """
    # Validate inputs
    if len(x) != len(y):
        raise ValueError("x and y coordinates must have the same length")
    if labels is not None and len(labels) != len(x):
        raise ValueError("labels must have the same length as coordinates")
    
    # Initialize LabelMe JSON structure
    labelme_json = {
        "version": "5.0.1",
        "flags": {},
        "shapes": [],
        "imagePath": os.path.basename(image_path) if image_path else "",
        "imageData": None,  # LabelMe stores base64 image data here, but we'll leave it empty
        "imageHeight": image_height,
        "imageWidth": image_width
    }
    
    # Convert each point to LabelMe shape
    point_size = 5  # Size of the point representation in pixels
    
    for i in range(len(x)):
        # Skip if coordinates are NaN
        if pd.isna(x[i]) or pd.isna(y[i]):
            continue
            
        shape = {
            "label": str(labels.iloc[i]) if labels is not None else "point",
            "points": [
                [float(x[i]) - point_size, float(y[i]) - point_size],  # Top-left
                [float(x[i]) + point_size, float(y[i]) + point_size]   # Bottom-right
            ],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        }
        
        labelme_json['shapes'].append(shape)
    
    # Add creation time
    labelme_json['timeStamp'] = datetime.now().isoformat()
    
    return labelme_json

def save_labelme_json(labelme_json, output_path):
    """Save the LabelMe JSON to file."""
    with open(output_path, 'w') as f:
        json.dump(labelme_json, f, indent=2)
        

## Using the csv input file function

In [3]:
# Example with pandas DataFrame
df = pd.DataFrame({
    'longitude': [120.5, 120.6, 120.7],
    'latitude': [-30.1, -30.2, -30.3],
    'labels': ['tree', 'building', 'car']
})
    
# Convert and save
labelme_json = csv_to_labelme(
    x=df['longitude'],
    y=df['latitude'],
    labels=df['labels'],
    image_path="image.jpg",
    image_height=1080,
    image_width=1920
)
    
save_labelme_json(labelme_json, "images/labels/labelme_annotation_test.json")